In [1]:
import thermoextrap.xtrapy.xpan_beta as xpan_beta
import xarray as xr
import numpy as np
import thermoextrap.xtrapy.core as xtrapy_core

In [2]:
def stack_xr_for_gp(da, xdims, ydims=None, 
                    xname='xstack',
                    yname='ystack',
                    variable=None,
                    policy='raise'):
    """
    Given an xarray.DataArray, stack into new shape for analysis
    
    Parameters
    ----------
    dims_x : str or tuple of strings
        dimensions to consider for X
    dims_y : str, tuple of strings, optional
        if passed, these dimensions are stacked.  If not passed,
        use all remaining dimensions
    variable : str or tuple of strings, optional
        use this to indicate a dimension that contains variables (mean and variance).
        This dimenension is moved to the last position.
    policy : {'raise', 'infer'}
        policy if coordinates not available
    
    Returns
    -------
    x : array, shape=(N, D)
        feature array to consider.  this will be in the order specified by dims_x
    y : array, sahpe=(N, P)
    
    
    """
    dims = da.dims
    for name in [xname, yname]:
        if name in dims:
            raise ValueError('{} conficts with existing {}'.format(xname, dims))
    
    if isinstance(xdims, str):
        xdims = (xdims,)
        
    stacker = {xname : xdims}
    if isinstance(ydims, str):
        ydims = (ydims,)
    elif ydims is None:
        ydims = tuple(set(dims) - set(xdims))
    
    if len(ydims) > 0:
        stacker[yname] = ydims
        
    if policy == 'raise':
        for dim in xdims:
            if dim not in da.coords:
                raise ValueError('da.coords[{}] not set'.format(dim))
    
    out = da.stack(**stacker)
    
    if variable is not None:
        if isinstance(variable, str):
            variable = (variable,)
        out = out.transpose(...,*variable)
    
    return out

def wrap_like(x, da):
    """
    wrap an array x with properties of da
    """
    return xr.DataArray(x, dims=da.dims, coords=da.coords, indexes=da.indexes, attrs=da.attrs, name=da.name)


def get_x_from_stack(stack, xname='xstack'):
    """
    get numpy array of X coordinates
    """
    da = stack[xname]
    out = np.array(list(da.values))
    return out



import pandas as pd
def to_ave_var(da, dim_reduce, dim_concat=None, concat_kws=None, reduce_kws=None):
    """
    for a dataarray apply average/variance along a dimension
    
    new array has dimension `dim_concat` which denotes mean and variance
    
    """
    if reduce_kws is None:
        reduce_kws = {}
    if concat_kws is None:
        concat_kws = {}
        
    if dim_concat is None:
        dim_concat = pd.Index(['mean','var'], name='variable')
    
    return xr.concat(
        (da.mean(dim_reduce, **reduce_kws), 
         da.var(dim_reduce, **reduce_kws)), 
        dim=dim_concat, **concat_kws)
    
    
    

In [55]:
# synthetic data
# pretending data for rdf(pair, r)
# here, pair is for a-a, a-b, b-b
shape = (3, 2, 4)
dims = ['rec', 'pair', 'position']

coords = {'position': np.linspace(0, 2, shape[-1]), 'pair': ['a','b']}

xems = []
for beta in [0.1, 10.0]:
    x = xr.DataArray(np.random.rand(*shape), dims=dims, coords=coords)
    u = xr.DataArray(np.random.rand(shape[0]), dims=dims[0])
    data = xpan_beta.DataCentralMomentsVals.from_vals(x, u, order=3, central=True)
    xems.append(xpan_beta.factory_extrapmodel(beta, data))

s = xtrapy_core.StateCollection(xems)

In [100]:
# resample the object to take mean/var
sr = s.resample(nrep=100)
# create Y data to consider:
Y_unstack = xr.concat(
    (x.xcoefs(norm=False).pipe(to_ave_var, 'rep') for x in sr), 
    dim=pd.Index(sr.alpha0, name=sr.alpha_name))

In [106]:
from importlib import reload

In [119]:
import thermoextrap.xtrapy.xstack as xstack
reload(xstack)

<module 'thermoextrap.xtrapy.xstack' from '/Users/wpk/Documents/python/projects/thermodynamic-extrapolation/thermoextrap/xtrapy/xstack.py'>

In [120]:
out = xstack.states_xcoefs_concat(sr).pipe(xstack.to_mean_var, 'rep').pipe(xstack.stack_dataarray, xdims=['beta','order'], vdim='variable')

In [125]:
X = xstack.multiindex_to_array(out.indexes['xstack'])

In [95]:
def _states():
    shape = (3, 2, 4)
    dims = ['rec', 'pair', 'position']

    coords = {'position': np.linspace(0, 2, shape[-1])}

    xems = []
    for beta in [0.1, 10.0]:
        x = xr.DataArray(np.random.rand(*shape), dims=dims, coords=coords)
        u = xr.DataArray(np.random.rand(shape[0]), dims=dims[0])
        data = xpan_beta.DataCentralMomentsVals.from_vals(x, u, order=3, central=True)
        xems.append(xpan_beta.factory_extrapmodel(beta, data))
    s = xtrapy_core.StateCollection(xems)

    return s.resample(nrep=3)


def test_mean_var():
    states = _states()

    x = states[0].xcoefs(norm=False)

    out = xstack.to_mean_var(x, dim='rep')
    xr.testing.assert_allclose(out.sel(variable='mean', drop=True), x.mean('rep'))

    xr.testing.assert_allclose(out.sel(variable='var', drop=True), x.var('rep'))


In [96]:
test_mean_var()

In [92]:
xr.testing.assert_allclose(out.sel(variable='mean', drop=True), x.mean('rep'))

In [80]:
a = xstack.to_mean_var(sr[0].xcoefs(norm=False), dim='rep').sel(variable='mean', drop=True)

In [81]:
b = sr[0].xcoefs(norm=False).mean('rep')

In [82]:
xr.testing.assert_allclose(a, b)

In [69]:
type(Y.indexes['ystack'])

pandas.core.indexes.multi.MultiIndex

In [60]:
Y_unstack.sel(variable='mean', drop=True)

<xarray.DataArray (beta: 2, order: 4, pair: 2, position: 4)>
array([[[[ 4.30354702e-01,  6.05672151e-01,  4.51925489e-01,
           5.34854433e-01],
         [ 7.06450406e-01,  6.14915778e-01,  4.59612928e-01,
           5.24214360e-01]],

        [[-2.21241182e-02,  2.07490712e-03, -1.98689793e-03,
           1.33729478e-02],
         [-3.84794679e-02,  9.24825966e-03,  9.23144572e-03,
          -2.81559801e-02]],

        [[-5.06743130e-04,  2.75130821e-04, -2.63981309e-04,
           4.29865241e-04],
         [-1.16793471e-03,  2.51955151e-05,  4.39427755e-05,
          -9.61847615e-04]],

        [[ 8.26495347e-04, -8.73292712e-05,  8.36475614e-05,
          -5.04905274e-04],
         [ 1.44984562e-03, -3.37439854e-04, -3.37636902e-04,
           1.06549866e-03]]],


       [[[ 7.19177562e-01,  6.26420664e-01,  5.82910377e-01,
           8.54863886e-01],
         [ 6.53186975e-01,  5.39226158e-01,  4.38635202e-01,
           5.60392588e-01]],

        [[-3.69695577e-03,  4.23107585e-02,  3.99619683e-02,
           8.64940953e-03],
         [ 1.62907132e-02,  4.10919337e-02,  2.07026615e-02,
           1.79579643e-02]],

        [[ 1.07259088e-03, -6.45132480e-04, -8.25725874e-04,
          -2.08003569e-04],
         [ 9.13826590e-04, -2.20000079e-04,  1.12892431e-04,
           6.84087473e-04]],

        [[ 8.01950203e-05, -2.16034925e-03, -2.01730231e-03,
          -4.33498574e-04],
         [-9.55955019e-04, -2.14155083e-03, -1.10284411e-03,
          -1.01925504e-03]]]])
Coordinates:
  * position  (position) float64 0.0 0.6667 1.333 2.0
  * pair      (pair) <U1 'a' 'b'
  * beta      (beta) float64 0.1 10.0
Dimensions without coordinates: order

In [57]:
Y = stack_xr_for_gp(Y_unstack, xdims=['beta','order'], ydims=['pair','position'], policy='infer', variable='variable')

In [59]:
Y.indexes

xstack: MultiIndex([( 0.1, 0),
                    ( 0.1, 1),
                    ( 0.1, 2),
                    ( 0.1, 3),
                    (10.0, 0),
                    (10.0, 1),
                    (10.0, 2),
                    (10.0, 3)],
                   names=['beta', 'order'])
ystack: MultiIndex([('a',                0.0),
                    ('a', 0.6666666666666666),
                    ('a', 1.3333333333333333),
                    ('a',                2.0),
                    ('b',                0.0),
                    ('b', 0.6666666666666666),
                    ('b', 1.3333333333333333),
                    ('b',                2.0)],
                   names=['pair', 'position'])
variable: Index(['mean', 'var'], dtype='object', name='variable')

In [29]:
a = list(Y_unstack.dims)
a

['beta', 'variable', 'order', 'pair', 'position']

In [30]:
a.remove('beta')

In [39]:
xr.concat?

In [38]:
np.array(list(Y.indexes['xstack'].values))

array([[ 0.1,  0. ],
       [ 0.1,  1. ],
       [ 0.1,  2. ],
       [ 0.1,  3. ],
       [10. ,  0. ],
       [10. ,  1. ],
       [10. ,  2. ],
       [10. ,  3. ]])

In [25]:
a

{'order', 'pair', 'position', 'variable'}

In [16]:
Y

<xarray.DataArray (xstack: 8, ystack: 15, variable: 2)>
array([[[ 4.37237841e-01,  4.33538725e-03],
        [ 5.44869023e-01,  4.51279546e-03],
        [ 3.70584327e-01,  2.89637202e-03],
        [ 4.56906627e-01,  2.92275097e-03],
        [ 4.57507067e-01,  4.95289884e-03],
        [ 4.79034071e-01,  3.38555007e-03],
        [ 4.71940320e-01,  4.09056400e-03],
        [ 6.23179338e-01,  3.77296917e-03],
        [ 5.26429208e-01,  4.51046710e-03],
        [ 4.10896119e-01,  4.01886753e-03],
        [ 4.90270208e-01,  3.06088654e-03],
        [ 5.90470295e-01,  7.21435540e-03],
        [ 3.93713371e-01,  2.44223225e-03],
        [ 5.68242309e-01,  5.03941445e-03],
        [ 5.83132318e-01,  4.09270928e-03]],

       [[-1.63841528e-02,  3.95053186e-04],
        [-1.74437079e-02,  3.41952899e-04],
        [-1.47133645e-02,  1.65227115e-04],
        [-2.61469812e-03,  3.75136459e-04],
...
        [-1.17290062e-03,  3.19682089e-05],
        [-1.01125598e-03,  2.83884081e-05],
        [ 6.66694457e-04,  6.67198172e-05],
        [ 8.55040324e-04,  2.29685520e-05]],

       [[-1.00414115e-04,  6.02228096e-06],
        [ 2.11739194e-03,  1.17298319e-05],
        [ 2.38943916e-03,  9.26570808e-06],
        [ 1.48082040e-03,  7.84739741e-06],
        [-1.40712428e-03,  7.44016871e-06],
        [ 2.83437323e-04,  5.84562538e-06],
        [ 1.17538592e-03,  7.23715820e-06],
        [-6.80493367e-04,  9.16619459e-06],
        [ 1.62363081e-03,  1.41913338e-05],
        [ 3.26797769e-03,  8.82053244e-06],
        [ 1.88598416e-03,  9.23733243e-06],
        [ 9.98292285e-04,  1.02223145e-05],
        [ 2.45458879e-03,  8.32398824e-06],
        [ 6.44625513e-03,  1.73088084e-05],
        [ 5.38324077e-04,  7.70660706e-06]]])
Coordinates:
  * variable  (variable) object 'mean' 'var'
  * xstack    (xstack) MultiIndex
  - beta      (xstack) float64 0.1 0.1 0.1 0.1 10.0 10.0 10.0 10.0
  - order     (xstack) int64 0 1 2 3 0 1 2 3
  * ystack    (ystack) MultiIndex
  - pair      (ystack) int64 0 0 0 0 0 1 1 1 1 1 2 2 2 2 2
  - position  (ystack) float64 0.0 0.5 1.0 1.5 2.0 0.0 ... 0.0 0.5 1.0 1.5 2.0

In [9]:
X = get_x_from_stack(Y)

In [67]:
for meta, y in Y.groupby('ystack'):
    print('pair: {}, position: {}'.format(*meta))
    break

pair: 0, position: 0.0


In [77]:
Y.indexes['xstack'].names

FrozenList(['beta', 'order'])

In [ ]:
np.random.rand(100)

In [ ]:
def wrap_X_input_array():
    """provide an array x with same shape as target X"""
    pass